### Loading Libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPool2D, Dropout, Flatten
from sklearn.metrics import accuracy_score,precision_score,confusion_matrix
from sklearn.preprocessing import normalize


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import cv2
import glob 
import math 
import shutil
import os
import time
import json
plt.rcParams['figure.figsize'] = [15, 5]
base_log_dir = "./logs/"
sns.set_style("darkgrid")

if not os.path.exists(base_log_dir):
    os.makedirs(base_log_dir)

### Plotting utils

In [ ]:
def CalculateMetricsAndPlot(true_label, predicted_label,color="Blues",text=""):
    plt.rcParams['figure.figsize'] = [9, 6]
    CM = confusion_matrix(true_label, predicted_label)
    acc = round(accuracy_score(true_label,predicted_label)*100,2)
    precision = round(precision_score(true_label,predicted_label, average='macro'),2)
    if text == "":
        sns.heatmap(CM ,annot=True, cmap=color, fmt='g').set_title("Confusion Matrix for Test Data | Accuracy={0}% | Precision={1}".format(acc,precision))
    else :
        sns.heatmap(CM ,annot=True, cmap=color, fmt='g').set_title("Confusion Matrix for Test Data | Accuracy={0}% | Precision={1} | {2}".format(acc,precision,text))
    
    plt.show()

def TrainingPlot(_history,text=None):
    plt.rcParams['figure.figsize'] = [15, 5]
    plt.subplot(1,2,1)
    plt.plot(_history["loss"],label="Train loss")
    plt.plot(_history["val_loss"],label="Validation loss")
    if not text is None:
        plt.title("Loss/Epoch | {0}".format(text))
    else:
        plt.title("Loss/Epoch")
    plt.xlabel("Epoch")
    plt.ylabel("Sparse Categorical Crossentropy")
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(_history["accuracy"],label="Train accuracy")
    plt.plot(_history["val_accuracy"],label="Validation accuracy")
    if not text is None:
        plt.title("Accuracy/Epoch | {0}".format(text))
    else:
        plt.title("Accuracy/Epoch")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy in %")
    plt.legend()
  
    plt.tight_layout()
    plt.show()


### Loading Dataset

In [ ]:
import tensorflow_datasets as tfds

(X_train, Y_train), (X_test, Y_test) , (X_valid, Y_valid) =  tfds.as_numpy(tfds.load('beans', 
    split = ['train', 'test','validation'], 
    batch_size=-1, 
    as_supervised=True))


### Preprocessing on dadtaset

In [ ]:
def ConvertDataset(Size,Chnl):
    _X_train = np.zeros((X_train.shape[0],Size,Size,Chnl))
    _X_test = np.zeros((X_test.shape[0],Size,Size,Chnl))
    _X_valid = np.zeros((X_valid.shape[0],Size,Size,Chnl))

    for index in range(X_train.shape[0]):
        tmp = cv2.resize(np.array(X_train[index]),(Size,Size),interpolation=cv2.INTER_AREA)
        if Chnl == 1:
          tmp = cv2.cvtColor(tmp,cv2.COLOR_RGB2GRAY)
        _X_train[index,:,:,:] = np.reshape(tmp,(Size,Size,Chnl))

    for index in range(X_test.shape[0]):
        tmp = cv2.resize(np.array(X_test[index]),(Size,Size),interpolation=cv2.INTER_AREA)
        if Chnl == 1:
          tmp = cv2.cvtColor(tmp,cv2.COLOR_RGB2GRAY)
        _X_test[index,:,:,:] = np.reshape(tmp,(Size,Size,Chnl))

    for index in range(X_valid.shape[0]):
        tmp = cv2.resize(np.array(X_valid[index]),(Size,Size),interpolation=cv2.INTER_AREA)
        if Chnl == 1:
          tmp = cv2.cvtColor(tmp,cv2.COLOR_RGB2GRAY)
        _X_valid[index,:,:,:] = np.reshape(tmp,(Size,Size,Chnl))

    return _X_train,_X_test, _X_valid

### Functions to make primary and custom LeNet

In [ ]:
def LeNetMaker(input_size=(32,32,1)):
    model = Sequential()
    model.add(keras.layers.InputLayer(input_size))
    model.add(keras.layers.Rescaling(1./255))
    model.add(keras.layers.Conv2D(6,kernel_size=(5,5),strides=(1,1),activation="tanh", padding='valid'))
    model.add(keras.layers.AveragePooling2D((2,2),strides=2))

    model.add(keras.layers.Conv2D(16,kernel_size=(5,5),strides=(1,1),activation="tanh", padding='valid'))
    model.add(keras.layers.AveragePooling2D((2,2),strides=2))

    model.add(keras.layers.Conv2D(120,kernel_size=(5,5),strides=(1,1),activation="tanh", padding='valid'))

    model.add(keras.layers.Flatten())

    model.add(keras.layers.Dense(84,activation="tanh"))

    model.add(keras.layers.Dense(3,activation="softmax"))

    return model 

def LeNetMakerWithRelu(input_size=(32,32,1)):
    model = Sequential()
    model.add(keras.layers.InputLayer(input_size))
    model.add(keras.layers.Rescaling(1./255))
    model.add(keras.layers.Conv2D(6,kernel_size=(5,5),strides=(1,1),activation="relu", padding='valid'))
    model.add(keras.layers.AveragePooling2D((2,2),strides=2))

    model.add(keras.layers.Conv2D(16,kernel_size=(5,5),strides=(1,1),activation="relu", padding='valid'))
    model.add(keras.layers.AveragePooling2D((2,2),strides=2))

    model.add(keras.layers.Conv2D(120,kernel_size=(5,5),strides=(1,1),activation="relu", padding='valid'))

    model.add(keras.layers.Flatten())

    model.add(keras.layers.Dense(84,activation="relu"))

    model.add(keras.layers.Dense(3,activation="softmax"))

    return model 

def CustomLeNetMaker(input_size=(32,32,3),drop_out=None,regularization=None,C1_n=6,C1_s=5,C2_n=16,C2_s=5,C3_n=120,C3_s=5,fc=84,activation="tanh"):
    model = Sequential()
    model.add(keras.layers.InputLayer(input_size))

    if regularization is None :
      model.add(keras.layers.Conv2D(C1_n,kernel_size=(C1_s,C1_s),strides=(1,1),activation=activation, padding='valid'))
    else:
      model.add(keras.layers.Conv2D(C1_n,kernel_size=(C1_s,C1_s),strides=(1,1),activation=activation, padding='valid',kernel_regularizer=regularization))

    if not drop_out is None:
      model.add(keras.layers.Dropout(drop_out))

    model.add(keras.layers.AveragePooling2D((2,2),strides=2))   

    if regularization is None :
      model.add(keras.layers.Conv2D(C2_n,kernel_size=(C2_s,C2_s),strides=(1,1),activation=activation, padding='valid'))
    else:
      model.add(keras.layers.Conv2D(C2_n,kernel_size=(C2_s,C2_s),strides=(1,1),activation=activation, padding='valid',kernel_regularizer=regularization))
    
    if not drop_out is None:
      model.add(keras.layers.Dropout(drop_out))

    model.add(keras.layers.AveragePooling2D((2,2),strides=2))   

    if regularization is None :
      model.add(keras.layers.Conv2D(C3_n,kernel_size=(C3_s,C3_s),strides=(1,1),activation=activation, padding='valid'))
    else:
      model.add(keras.layers.Conv2D(C3_n,kernel_size=(C3_s,C3_s),strides=(1,1),activation=activation, padding='valid',kernel_regularizer=regularization))
    
    if not drop_out is None:
      model.add(keras.layers.Dropout(drop_out))


    model.add(keras.layers.Flatten())

    model.add(keras.layers.Dense(fc,activation=activation))

    model.add(keras.layers.Dense(3,activation="softmax"))

    return model 


### An Inception Transfering | Train All freezed layers after classifier

In [ ]:
_X_train,_X_test,_X_valid = ConvertDataset(299,3)

inception_base = keras.applications.InceptionV3(weights="imagenet", input_shape=(299, 299, 3),include_top=False)
inception_base.trainable = False

inputs = keras.Input(shape=(299, 299, 3))
scaling = keras.layers.Rescaling(scale=1. /127.5,offset=-1)
scaling = scaling(inputs)

transfer = inception_base(scaling, training=False)
transfer = keras.layers.GlobalAveragePooling2D()(transfer)
outputs = keras.layers.Dense(32,activation="relu")(transfer)
outputs = keras.layers.Dense(3,activation="softmax")(outputs)
model = keras.Model(inputs, outputs)

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
History = model.fit(_X_train, Y_train, epochs=16, validation_data=(_X_valid,Y_valid),shuffle=True,verbose=0)
CalculateMetricsAndPlot(Y_test,np.argmax(model.predict(_X_test),axis=1),color="Blues",text="Training Classifier")
TrainingPlot(History.history,text="Training Classifier")


inception_base.trainable = True
model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-4), metrics=["accuracy"])
History = model.fit(_X_train, Y_train, epochs=8, validation_data=(_X_valid,Y_valid),shuffle=True,verbose=0)
CalculateMetricsAndPlot(Y_test,np.argmax(model.predict(_X_test),axis=1),color="Purples",text="Training with All Inception Layers")
TrainingPlot(History.history,text="Training with All Inception Layers")


keras.utils.plot_model(model,to_file="inception.png",show_layer_activations=True,show_shapes=True,show_layer_names=False)

### Extract Concatenate indexes in inception

In [ ]:
a= ["Concatenate" in str(inception_base.layers[i]) for i in range(311)]
[i for i, x in enumerate(a) if x == True][::-1]

### One Inception module trainable

In [ ]:

inception_base = keras.applications.InceptionV3(weights="imagenet", input_shape=(299, 299, 3),include_top=False)
inception_base.trainable = False

inputs = keras.Input(shape=(299, 299, 3))
scaling = keras.layers.Rescaling(scale=1. /127.5,offset=-1)
scaling = scaling(inputs)

transfer = inception_base(scaling, training=False)
transfer = keras.layers.GlobalAveragePooling2D()(transfer)
outputs = keras.layers.Dense(32,activation="relu")(transfer)
outputs = keras.layers.Dense(3,activation="softmax")(outputs)
model = keras.Model(inputs, outputs)

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
History = model.fit(_X_train, Y_train, epochs=16, validation_data=(_X_valid,Y_valid),shuffle=True,verbose=0)
CalculateMetricsAndPlot(Y_test,np.argmax(model.predict(_X_test),axis=1),color="Blues",text="Training Classifier")
TrainingPlot(History.history,text="Training Classifier")

for i in range(280,311):
  inception_base.layers[i].trainable = True

model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-4), metrics=["accuracy"])
History = model.fit(_X_train, Y_train, epochs=8, validation_data=(_X_valid,Y_valid),shuffle=True,verbose=0)
CalculateMetricsAndPlot(Y_test,np.argmax(model.predict(_X_test),axis=1),color="Purples",text="Training with last one Inception modules")
TrainingPlot(History.history,text="Training with last one Inception modules")


### Two Inception module trainable

In [ ]:

inception_base = keras.applications.InceptionV3(weights="imagenet", input_shape=(299, 299, 3),include_top=False)
inception_base.trainable = False

inputs = keras.Input(shape=(299, 299, 3))
scaling = keras.layers.Rescaling(scale=1. /127.5,offset=-1)
scaling = scaling(inputs)

transfer = inception_base(scaling, training=False)
transfer = keras.layers.GlobalAveragePooling2D()(transfer)
outputs = keras.layers.Dense(32,activation="relu")(transfer)
outputs = keras.layers.Dense(3,activation="softmax")(outputs)
model = keras.Model(inputs, outputs)

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
History = model.fit(_X_train, Y_train, epochs=16, validation_data=(_X_valid,Y_valid),shuffle=True,verbose=0)
CalculateMetricsAndPlot(Y_test,np.argmax(model.predict(_X_test),axis=1),color="Blues",text="Training Classifier")
TrainingPlot(History.history,text="Training Classifier")

for i in range(249,311):
  inception_base.layers[i].trainable = True

model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-4), metrics=["accuracy"])
History = model.fit(_X_train, Y_train, epochs=8, validation_data=(_X_valid,Y_valid),shuffle=True,verbose=0)
CalculateMetricsAndPlot(Y_test,np.argmax(model.predict(_X_test),axis=1),color="Purples",text="Training with last Two Inception modules")
TrainingPlot(History.history,text="Training with last Two Inception modules")


### Three Inception module trainable

In [ ]:

inception_base = keras.applications.InceptionV3(weights="imagenet", input_shape=(299, 299, 3),include_top=False)
inception_base.trainable = False

inputs = keras.Input(shape=(299, 299, 3))
scaling = keras.layers.Rescaling(scale=1. /127.5,offset=-1)
scaling = scaling(inputs)

transfer = inception_base(scaling, training=False)
transfer = keras.layers.GlobalAveragePooling2D()(transfer)
outputs = keras.layers.Dense(32,activation="relu")(transfer)
outputs = keras.layers.Dense(3,activation="softmax")(outputs)
model = keras.Model(inputs, outputs)

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
History = model.fit(_X_train, Y_train, epochs=16, validation_data=(_X_valid,Y_valid),shuffle=True,verbose=0)
CalculateMetricsAndPlot(Y_test,np.argmax(model.predict(_X_test),axis=1),color="Blues",text="Training Classifier")
TrainingPlot(History.history,text="Training Classifier")

for i in range(229,311):
  inception_base.layers[i].trainable = True

model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-4), metrics=["accuracy"])
History = model.fit(_X_train, Y_train, epochs=8, validation_data=(_X_valid,Y_valid),shuffle=True,verbose=0)
CalculateMetricsAndPlot(Y_test,np.argmax(model.predict(_X_test),axis=1),color="Purples",text="Training with last Three Inception modules")
TrainingPlot(History.history,text="Training with last Three Inception modules")


### Four Inception module trainable

In [ ]:

inception_base = keras.applications.InceptionV3(weights="imagenet", input_shape=(299, 299, 3),include_top=False)
inception_base.trainable = False

inputs = keras.Input(shape=(299, 299, 3))
scaling = keras.layers.Rescaling(scale=1. /127.5,offset=-1)
scaling = scaling(inputs)

transfer = inception_base(scaling, training=False)
transfer = keras.layers.GlobalAveragePooling2D()(transfer)
outputs = keras.layers.Dense(32,activation="relu")(transfer)
outputs = keras.layers.Dense(3,activation="softmax")(outputs)
model = keras.Model(inputs, outputs)

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
History = model.fit(_X_train, Y_train, epochs=16, validation_data=(_X_valid,Y_valid),shuffle=True,verbose=0)
CalculateMetricsAndPlot(Y_test,np.argmax(model.predict(_X_test),axis=1),color="Blues",text="Training Classifier")
TrainingPlot(History.history,text="Training Classifier")

for i in range(197,311):
  inception_base.layers[i].trainable = True

model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-4), metrics=["accuracy"])
History = model.fit(_X_train, Y_train, epochs=8, validation_data=(_X_valid,Y_valid),shuffle=True,verbose=0)
CalculateMetricsAndPlot(Y_test,np.argmax(model.predict(_X_test),axis=1),color="Purples",text="Training with last Four Inception modules")
TrainingPlot(History.history,text="Training with last Four Inception modules")


### Five Inception module trainable

In [ ]:

inception_base = keras.applications.InceptionV3(weights="imagenet", input_shape=(299, 299, 3),include_top=False)
inception_base.trainable = False

inputs = keras.Input(shape=(299, 299, 3))
scaling = keras.layers.Rescaling(scale=1. /127.5,offset=-1)
scaling = scaling(inputs)

transfer = inception_base(scaling, training=False)
transfer = keras.layers.GlobalAveragePooling2D()(transfer)
outputs = keras.layers.Dense(32,activation="relu")(transfer)
outputs = keras.layers.Dense(3,activation="softmax")(outputs)
model = keras.Model(inputs, outputs)

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
History = model.fit(_X_train, Y_train, epochs=16, validation_data=(_X_valid,Y_valid),shuffle=True,verbose=0)
CalculateMetricsAndPlot(Y_test,np.argmax(model.predict(_X_test),axis=1),color="Blues",text="Training Classifier")
TrainingPlot(History.history,text="Training Classifier")

for i in range(165,311):
  inception_base.layers[i].trainable = True

model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-4), metrics=["accuracy"])
History = model.fit(_X_train, Y_train, epochs=8, validation_data=(_X_valid,Y_valid),shuffle=True,verbose=0)
CalculateMetricsAndPlot(Y_test,np.argmax(model.predict(_X_test),axis=1),color="Purples",text="Training with last Five Inception modules")
TrainingPlot(History.history,text="Training with last Five Inception modules")


### Six Inception module trainable

In [ ]:

inception_base = keras.applications.InceptionV3(weights="imagenet", input_shape=(299, 299, 3),include_top=False)
inception_base.trainable = False

inputs = keras.Input(shape=(299, 299, 3))
scaling = keras.layers.Rescaling(scale=1. /127.5,offset=-1)
scaling = scaling(inputs)

transfer = inception_base(scaling, training=False)
transfer = keras.layers.GlobalAveragePooling2D()(transfer)
outputs = keras.layers.Dense(32,activation="relu")(transfer)
outputs = keras.layers.Dense(3,activation="softmax")(outputs)
model = keras.Model(inputs, outputs)

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
History = model.fit(_X_train, Y_train, epochs=16, validation_data=(_X_valid,Y_valid),shuffle=True,verbose=0)
CalculateMetricsAndPlot(Y_test,np.argmax(model.predict(_X_test),axis=1),color="Blues",text="Training Classifier")
TrainingPlot(History.history,text="Training Classifier")

for i in range(133,311):
  inception_base.layers[i].trainable = True

model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-4), metrics=["accuracy"])
History = model.fit(_X_train, Y_train, epochs=8, validation_data=(_X_valid,Y_valid),shuffle=True,verbose=0)
CalculateMetricsAndPlot(Y_test,np.argmax(model.predict(_X_test),axis=1),color="Purples",text="Training with last Six Inception modules")
TrainingPlot(History.history,text="Training with last Six Inception modules")
